In [4]:
from pprint import pprint
import itertools

input_file = "inputs/aoc_input_17.txt"
max_turns = 6

DEAD = '.'
ALIVE = '#'

In [5]:
if True:
    with open(input_file) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
else:
    content = '''
.#.
..#
###
'''.strip().split('\n')

In [6]:
def generate_2d_matrix(n):
    return [[DEAD for x in range(n)] for y in range(n)] 

def generate_nd_matrix(size, n):
    if n == 2:
        return generate_2d_matrix(size)
    return[generate_nd_matrix(size, n-1) for x in range(size)]


In [7]:
# This could be made much more efficient if the matrix was expanded only when necessary.
# Without using numpy or some other lib for matrixes, that would be a major pain in the ass
# So I just expand all it can possibly need to be expanded beforehand
def expand_matrix(m, D, turns):
    SIZE = len(m)
    
    max_matrix_size = SIZE + turns*2
    conway_matrix = generate_nd_matrix(max_matrix_size, D)
    
    dimension_padding = [turns + int(SIZE/2) for x in range(D-2)]

    for x in range(SIZE):
        for y in range(SIZE):
            set_in_matrix(conway_matrix, dimension_padding + [turns + x, turns + y], m[x][y])
            
    return conway_matrix

In [8]:
def get_from_matrix(m, coords):
    if len(coords) == 1:
        return m[coords[0]]
    return get_from_matrix(m[coords[0]], coords[1:])

def set_in_matrix(m, coords, value):
    if len(coords) == 1:
        m[coords[0]] = value
    else:
        set_in_matrix(m[coords[0]], coords[1:], value)

def get_immediate_surroundings(m, coords):
    D = len(coords)
    SIZE = len(m)
    
    surroundings = []
    relative_locations = itertools.product([-1,0,1], repeat=D)
    
    for location in relative_locations:
        new_coords = [coord + delta for coord, delta in zip(coords, location)]
        within_bounds = all([coord < SIZE and 0 <= coord for coord in new_coords])
        is_zero = all([l==0 for l in location])
        
        if within_bounds and not is_zero:
            surroundings.append(get_from_matrix(m, new_coords))
    return surroundings

def do_step(m, D):
    SIZE = len(m)
    
    new_state = generate_nd_matrix(len(m), D)
    coords = itertools.product(range(SIZE), repeat=D)
    
    for coord in coords:
        cube_state = get_from_matrix(m, coord)
        surroundings = get_immediate_surroundings(m, coord)
        cubes_around = len([cube for cube in surroundings if cube == ALIVE])
        
        if cube_state == ALIVE:
            if cubes_around in [2, 3]:
                new_cube = ALIVE
            else:
                new_cube = DEAD
        else:
            if cubes_around == 3:
                new_cube = ALIVE
            else:
                new_cube = DEAD
                
        set_in_matrix(new_state, coord, new_cube)
    return new_state

def reduce_matrix(m, new_n):
    if not isinstance(m, list):
        return m
    
    new_m = []
    for i in range(int((len(m) - new_n)/2), int((len(m) + new_n)/2)):
        new_m.append(reduce_matrix(m[i], new_n))
        
    return new_m

In [9]:
def print_3d_matrix(m):
    for i,zs in enumerate(m):
        print('z =', i)
        for ys in zs:
            print(''.join(ys))
        print()

def print_2d_matrix(m):
        for ys in m:
            print(''.join(ys))
        print()


In [10]:
def run_conway(init_matrix, D, max_turns):
    m = expand_matrix(init_matrix, D, max_turns)
    for i in range(max_turns):
        m = do_step(m, D)
        
    return m

In [11]:
def count_alive(m, D):
    coords = itertools.product(range(len(m)), repeat=D)
    c = 0
    for coord in coords:
        if get_from_matrix(m, coord) == ALIVE:
            c += 1
    return c

In [12]:
conway_3 = run_conway(content, 3, max_turns)
count_alive(conway_3, 3)

291

In [13]:
conway_4 = run_conway(content, 4, max_turns)
count_alive(conway_4, 4)

1524